## 5.5 Reduktionsmethoden

In [ ]:
import numpy as np

**Transformation in die Hessenberg-Gestalt**

In [ ]:
def reduktion_auf_hessenberg(A):
    n, m = A.shape
    for i in range(m - 2):
        v = A[i + 1:, i].copy()
        ei = np.zeros(n - i - 1, dtype=A.dtype)
        ei[0] = 1
        v += np.sign(v[0]) * np.linalg.norm(v) * ei
        v /= np.linalg.norm(v)
        A[i + 1:, i:] -= 2 * np.outer(v, np.inner(v.T, A[i + 1:, i:].T))
        A[:, i + 1:] -= 2 * np.outer(np.inner(A[:, i + 1:], v), v)
    return None

Wir betrachten die Matrix
$$A = \begin{pmatrix}
338 & -20 & -90 & 32 \\ -20 & 17 & 117 & 70
\\ -90 & 117 & 324 & -252 \\ 32 & 70 & -252 & 131
\end{pmatrix}.$$
Mit den Eigenwerten

In [ ]:
A = np.array([[338, -20, -90, 32],
              [-20, 17, 117, 70],
              [-90, 117, 324, -252],
              [32, 70, -252, 131]], dtype=np.double)
A2 = A.copy()
eig_vor = np.linalg.eig(A)[0]
print(eig_vor)

Mit Hilfe von `numpy` können wir überprüfen, ob sich die Eigenwerte durch die Reduktion verändert haben:

In [ ]:
reduktion_auf_hessenberg(A)
eig_nach = np.linalg.eig(A)[0]
print(f'Absoluter Fehler der Eigenwerte nach Reduktion = {np.linalg.norm(eig_vor - eig_nach):6.4e}')

**QR-Zerlegung von Hessenberg-Matrizen**

Für die effiziente Berechnung der Eigenwerte müssen wir noch die QR-Zerlegung mit Householder-Matrizen für den Spezialfall von Hessenberg-Matrizen anpassen:

In [ ]:
def qr_householder_fuer_hessenberg(A):
    n, m = A.shape
    dtype = A.dtype
    Q = np.identity(n, dtype=dtype)
    
    for i in range(m - 1):
        v = A[i: i + 2, i].copy()
        ei = np.zeros(2, dtype=dtype)
        ei[0] = 1.0
        v += np.sign(A[i,i]) * np.linalg.norm(v) * ei
        v /= np.linalg.norm(v)
        A[i:i + 2, i:] -= 2 * np.outer(v, np.inner(v.T, A[i:i + 2, i:].T))
        Q[:, i:i + 2] -= 2 * np.outer(np.inner(Q[:, i: i + 2], v), v)
    return Q

Damit sind wir nun in der Lage das QR-Verfahren nach der Reduktion auf die Hessenberggestalt zu implementieren:

In [ ]:
def eigenwerte_hessenberg_qr(A, k):
    reduktion_auf_hessenberg(A)
    for i in range(k):
        Q = qr_householder_fuer_hessenberg(A)
        A = A @ Q
    return np.diagonal(A)

#### Beispiel 5.30 (Eigenwertberechnung mit Reduktion und QR-Verfahren)

Angewandt auf unser obiges Beispiel, erhalten wir nach 10 Schritten die Eigenwerte

In [ ]:
A = A2.copy()
eig = np.linalg.eig(A)[0]
eig_qr = eigenwerte_hessenberg_qr(A, 10)
eig_qr = np.flip(np.sort(eig_qr))
print(eig_qr)

Daraus ergeben sich dir relativen Fehler:

In [ ]:
print((abs(eig - eig_qr) / abs(eig)))

Also ein maximaler relativer Fehler von ca. $1\%$. Der Fehler für $\lambda_3$ und $\lambda_4$ ist größer als der für $\lambda_1$ und $\lambda_2$, da diese Eigenwerte weniger separiert sind.